In [ ]:
import json
import boto3

client = boto3.client('iot')
def lambda_handler(event, context):
    print(event['body'])
    data = json.loads(event['body'])
    name = data['name']
    policyn = 'policy_'+name
    client_id = 'id_'+name
    policy_d ={"Version": "2012-10-17","Statement":[{"Effect": "Allow","Action": "iot:Receive",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topic/lwt/"+name,\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update/delta"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Subscribe",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/lwt/"+name,\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/get/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/get/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/accepted",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/rejected",\
        "arn:aws:iot:us-east-1:830472830062:topicfilter/$aws/things/"+name+"/shadow/update/delta"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Connect",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:client/"+client_id,\
        "arn:aws:iot:us-east-1:830472830062:client/"+client_id+"_1"\
      ]\
    },\
    {\
      "Effect": "Allow",\
      "Action": "iot:Publish",\
      "Resource": [\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/get",\
        "arn:aws:iot:us-east-1:830472830062:topic/$aws/things/"+name+"/shadow/update",\
        "arn:aws:iot:us-east-1:830472830062:topic/lwt/"+name
      ]\
    }]
    }
    policy = json.dumps(policy_d)   #先把字典转成json
    #print(name)
    #创建证书和密匙
    res1 = client.create_keys_and_certificate(setAsActive=True)
    #print(res1)
    #创建策略
    response = client.create_policy(
    policyName=policyn,
    policyDocument=policy
    )
    #创建事物
    res3 = client.create_thing(
    thingName=name
    )
    #把策略附加到证书
    res2 = client.attach_policy(
    policyName=policyn,
    target=res1['certificateArn']
    )
    #print(res3)
    #把证书附加到事物
    res4 = client.attach_thing_principal(
    thingName=name,
    principal=res1['certificateArn']
    )
    #创建规则
    # response = client.create_topic_rule(
    #     ruleName=name.replace("-","")+'_rule',
    #     topicRulePayload={"sql": "SELECT * FROM 'my/things/"+name+"/update'",\
    #         "ruleDisabled": False,\
    #         "description": "Turn my/things/ into $aws/things/",\
    #         "actions": [\
    #             {\
    #             "republish": {\
    #             "topic": "$$aws/things/"+name+"/shadow/update",\
    #             "roleArn": "arn:aws:iam::830472830062:role/my-iot-role"\
    #             }\
    #             }\
    #         ]\
           
    #     }
    # )
    res = {'key':res1['keyPair']['PrivateKey'],'pem':res1['certificatePem']}
   # print(res4)
    return {
        'statusCode': 200,
        'body': res
    }
   
